In [9]:
%load_ext autoreload
%autoreload 2

import os
import oddsapi
import requests
import joblib
import pandas as pd
import numpy as np
from common.utils.odds_utils import get_tournament_winner_odds, get_group_stage_odds, get_group_stage_OU
from common.utils.holdet_utils import get_holdet_data, get_play_off_opps, get_rw
from common.utils.simulation_utils import WorldCupTeam, WorldCupSim
from common.utils.group_stage import get_group_stage, playoffs
from common.utils.dict_utils import eng2dan
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
ROUND = 1
API_KEY = os.getenv("API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# load data
df_holdet = get_holdet_data()
df_winner = get_tournament_winner_odds(API_KEY)
df_group_stage = get_group_stage_odds(API_KEY)
df_get_group_stage_OU = get_group_stage_OU(API_KEY)
df_groups = pd.read_csv("common/ressources/df_groups.csv")[["Team","group"]]
df_group_stage = df_group_stage[['home_team','away_team','prob1','probx', 'prob2']].merge(df_groups.rename(columns =  {"Team":"home_team"}), on = "home_team")

In [11]:
_df_group_stage = get_group_stage(df_get_group_stage_OU.copy(), df_group_stage.copy(), ROUND, eng2dan)

In [12]:
df_rw = get_rw(df_holdet)
df_rw.rename(columns = {"Player_match": "name1"}, inplace = True)
print(df_holdet.shape)
df_holdet = df_holdet.merge(df_rw, on = "name1", how ="left")
print(df_holdet.shape)
df_holdet = df_holdet[['name1', 'name2', 'country', 'price', 'position', 'Player', 'Team', 'Opp', 'Pos', 'MIN','G', 'A', 'SOG', 'Y','R']]
print(df_holdet.shape)

(835, 10)
(835, 39)
(835, 15)


In [13]:
for round in [str(_) for _ in range(ROUND,4)]:
    df_temp1 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["home_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"home_team":"country"}), on = "country")
    df_temp1.rename(columns = {"prob1":f"prob_winning_{round}","probx":f"prob_draw_{round}","prob2":f"prob_lose_{round}","xGH":f"E_goals_{round}","xGA":f"E_goals_against_{round}"}, inplace = True)

    df_temp2 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["away_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"away_team":"country"}), on = "country")
    df_temp2.rename(columns = {"prob1":f"prob_lose_{round}","probx":f"prob_draw_{round}","prob2":f"prob_winning_{round}","xGA":f"E_goals_{round}","xGH":f"E_goals_against_{round}"}, inplace = True)

    df_temp_temp = pd.concat([df_temp1,df_temp2],axis=0)
    df_holdet = df_holdet.merge(df_temp_temp, on = ["name1","name2","country"])

In [15]:
df_winner  = playoffs(df_winner.copy(),_df_group_stage.copy(), eng2dan)
df_holdet = df_holdet.merge(df_winner[['country','R16_prob', 'QF_prob', 'SF_prob', 'Final_prob', 'W_prob', \
                                       'prob_to_win_R16', 'prob_to_win_QF', 'prob_to_win_SF','prob_to_win_Final', \
                                       'xG_R16', 'xG_QF', 'xG_SF', 'xG_Final','xGa_R16', 'xGa_QF', 'xGa_SF', 'xGa_Final']], on = "country")

df_holdet["selected"] = ~df_holdet['G'].isna()

In [23]:
for country in df_holdet["country"].unique():
    xG_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float).sum()
    xA_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float).sum()
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float)/xG_sum
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float)/xA_sum

    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_total"] = -5000
    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_round1"] = -5000
    for player in df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),"name1"]:
        points = 0
        goal_share = df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]
        assist_share = df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]
        yellow_prob = df_holdet.loc[df_holdet["name1"]==player,'Y'].astype(float).values[0]
        red_prob = df_holdet.loc[df_holdet["name1"]==player,'R'].astype(float).values[0]
        # group games
        loop = 4
        for i in range(ROUND,loop):
            points += 7000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_winning_{i}'].values[0]*25000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_draw_{i}'].values[0]*5000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_lose_{i}'].values[0]*-15000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*10000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0]*-10000*(1/(i-ROUND+1))
            points += yellow_prob * -20000*(1/(i-ROUND+1))
            points += yellow_prob*yellow_prob*-20000*(1/(i-ROUND+1))
            points += red_prob * -50000*(1/(i-ROUND+1))
            if i == loop:
                for idx, game in enumerate(["R16","QF","SF","Final"]):
                    prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                    factor = 1/(2*(loop+idx))
                    points += 7000*factor
                    points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*-10000*factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0]*25000*factor
                    points += prob*(1-df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0])*-15000*factor
                    points += prob*yellow_prob * -20000*factor
                    points += prob*yellow_prob*yellow_prob*-20000*factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0]*10000*factor
                    points += prob*red_prob * -50000*factor
                    
            if df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Mål":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*75000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*75000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Forsvar":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*50000*(1/(i-ROUND+1))
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*175000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*50000*factor
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*175000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Midtbane":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*150000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*150000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Angreb":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*125000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*125000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
            if i == ROUND:
                df_holdet.loc[df_holdet["name1"]==player,"E_points_round1"] = points
        df_holdet.loc[df_holdet["name1"]==player,"E_points_total"] = points

In [22]:
df_holdet.to_csv("common/ressources/df_holdet", index = False)

In [20]:
df_holdet.loc[df_holdet["country"]=="Ecuador",:].sort_values("G_share")

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,...,xG_Final,xGa_R16,xGa_QF,xGa_SF,xGa_Final,selected,G_share,A_share,E_points_total,E_points_round1
495,Pervis Estupinan,P. Estupinan,Ecuador,4000000.0,Forsvar,Pervis Estupinan,ECU,QAT,M/D,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.000000,0.076190,36527.773064,36606.842604
497,Alexander Dominguez,A. Dominguez,Ecuador,3500000.0,Mål,Alexander Dominguez Carabali,ECU,QAT,G,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.000000,0.009524,40882.387663,40919.971005
498,Gonzalo Plata,G. Plata,Ecuador,3500000.0,Midtbane,Gonzalo Plata,ECU,QAT,F/M,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.000000,0.000000,20475.178578,20550.118210
507,Carlos Gruezo,C. Gruezo,Ecuador,3000000.0,Midtbane,Carlos Gruezo,ECU,QAT,M,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.000000,0.000000,9180.781880,9350.118210
509,Moises Ramirez,M. Ramirez,Ecuador,3000000.0,Mål,Wellington Moises Ramirez Preciado,ECU,QAT,G,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.000000,0.009524,40662.550299,40701.971005
503,Piero Hincapie,P. Hincapie,Ecuador,3000000.0,Forsvar,Piero Hincapie,ECU,QAT,D,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.045161,0.095238,49894.423353,49979.761920
500,Felix 'Feliz' Torres,F. Torres,Ecuador,3500000.0,Forsvar,Felix Torres,ECU,QAT,D,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.051613,0.000000,46619.229815,46679.978569
496,Moises Caicedo,M. Caicedo,Ecuador,4000000.0,Midtbane,Moises Caicedo,ECU,QAT,M,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.064516,0.152381,49407.908323,49485.671183
508,Romario Ibarra,R. Ibarra,Ecuador,3000000.0,Midtbane,Romario Ibarra,ECU,QAT,F/M,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.161290,0.219048,87053.076088,87061.419432
501,Michael Estrada,M. Estrada,Ecuador,3500000.0,Angreb,Michael Estrada,ECU,QAT,F,90,...,0.153855,0.190235,0.140741,0.335749,0.140741,True,0.174194,0.438095,106730.959508,106701.807231


In [64]:
df_holdet

,name1,country,E_points_round1,position
366,Ferran Torres,Spanien,205639.318674,Angreb
78,Lionel Messi,Argentina,190175.717927,Angreb
365,Alvaro Morata,Spanien,189893.032395,Angreb
52,Kylian Mbappe,Frankrig,188154.784040,Angreb
112,Cody Gakpo,Holland,175286.260740,Angreb
131,Romelu Lukaku,Belgien,173546.665504,Angreb
130,Kevin De Bruyne,Belgien,171773.797750,Midtbane
0,Neymar Jr.,Brasilien,161221.583307,Angreb
26,Harry Kane,England,159929.607625,Angreb
53,Karim Benzema,Frankrig,156664.391130,Angreb
